In [12]:
import albumentations as A
import cv2
import numpy as np
from PIL import Image

# 이미지 파일 경로 설정 (업로드된 이미지 경로)
image_path = r'C:\Users\fun67\Desktop\lastpj\data\test\images\ff40483b59dc7c5005651a46.jpg'

# 이미지 읽기
image = cv2.imread(image_path)

# OpenCV가 이미지를 읽을 수 없을 경우 오류 처리
if image is None:
    raise FileNotFoundError(f"Image not found at {image_path}")

# 이미지 변환을 위해 BGR에서 RGB로 변환 (PIL 사용을 위해)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 이미지 증강 파이프라인 설정 (RandomSizedBBoxSafeCrop 제거)
max_size = 640
train_augmentation_and_transform = A.Compose(
    [
        A.ShiftScaleRotate(p=0.3),
        A.SmallestMaxSize(max_size=max_size, p=1.0),
        A.OneOf(
            [
                A.Blur(blur_limit=7, p=0.5),
                A.MotionBlur(blur_limit=7, p=0.5),
                A.Defocus(radius=(1, 5), alias_blur=(0.1, 0.25), p=0.1),
            ],
            p=0.2,
        ),
        A.Perspective(p=0.2),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.1),
        A.CLAHE(p=0.1),
        A.RGBShift(p=0.1),
        A.ChannelShuffle(p=0.1),
        A.Emboss(p=0.1),
    ]
)

# 여러 번 증강 적용
def apply_transformations(image, num_variations=40):
    images = []
    for _ in range(num_variations):
        augmented = train_augmentation_and_transform(image=image)['image']
        images.append(augmented)
    return images

# 증강된 이미지 목록 가져오기
augmented_images = apply_transformations(image, num_variations=40)

# 이미지를 하나의 큰 이미지로 결합하기 (5x8 그리드)
def create_collage(images, rows, cols):
    assert len(images) == rows * cols, "Rows * cols should equal the number of images"
    
    # 각 이미지 크기를 확인하여 결합할 캔버스 크기 계산
    width, height = images[0].shape[1], images[0].shape[0]
    collage = Image.new('RGB', (cols * width, rows * height))

    # 각 위치에 이미지를 배치
    for i, img in enumerate(images):
        img_pil = Image.fromarray(img)
        collage.paste(img_pil, ((i % cols) * width, (i // cols) * height))
    
    return collage

# 5x8 그리드로 이미지 결합
collage_image = create_collage(augmented_images, rows=5, cols=8)

# 결합된 이미지 보여주기
collage_image.show()
